In [ ]:
import numpy as np
import pandas as pd
from time import sleep
#pip install googlesearch-python
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
df=pd.read_excel('companies.xlsx')
df.head()

In [ ]:
# get the linkedin profile link for particular company from google search API
links=[]
counter=0
for company in df["Name"]:
    print(company)
    try: 
        from googlesearch import search 
    except ImportError:  
        print("No module named 'google' found")
    counter+=1
    print(counter)
    
    # write the search keyword here 
    search_query = company + ' linkedin'
    list_of_links=[]
  
    for link in search(search_query): 
        list_of_links.append(link)
    print(list_of_links)
    
    # we can take the most relevant link here from our search, which for now we take the first one
    links.append(list_of_links[0])
    #print(list_of_links)

In [ ]:
links

In [ ]:
def extractEmpNo(employeeText):
    # this function returns the employee number for the particular company from linkedin page
    indexEmployee = employeeText.find('employees')
    numEmployee = employeeText[8:indexEmployee]
    return numEmployee

def fetchEmployeeNumber(CompanyName):   
    print('Initiating Selenium ---')
    driver = webdriver.Chrome(ChromeDriverManager().install())
    # driver = webdriver.Chrome('give path to your file here')
    print('Opening LinkedIn ---')
    driver.get('https://www.linkedin.com')
    
    # signing in to linkedin...
    signInButtonElem = driver.find_element_by_link_text("Sign in")
    sleep(10)
    signInButtonElem.click()
    username = driver.find_element_by_id("username")
    sleep(5)
    username.send_keys('-- add username to login to linkedin here --')
    sleep(10)
    password = driver.find_element_by_id("password")
    sleep(5)
    password.send_keys('-- add relevant account password here --')
    sleep(10)
    log_in_button = driver.find_element_by_xpath('//*[@type="submit"]')
    sleep(10)
    log_in_button.click() 
    try:
        # in case there is phone verification, then skip it
        enter_phone_number = driver.find_element_by_class_name('//*[@id="ember983"]/button')
        enter_phone_number.click()
    except Exception as e:
        print('element not found'+str(e))
    sleep(10)
    companyName = CompanyName
    print(companyName)
    numEmployeeText = ''
    numEmp = 0 
    try:
        # open company linkedin page and scrape relevant information
        absoluteUrl=CompanyName
        driver.get(absoluteUrl)
        sleep(10)
        numEmployeeLink = driver.find_element_by_partial_link_text('See all')
        numEmployeeText = numEmployeeLink.text
        numEmp = int(extractEmpNo(numEmployeeText))
        print(numEmp)
        if(numEmp > 0):
            return str(numEmp) + '#' + str(absoluteUrl)   
    except Exception as e3:
        print(companyName + ' company does not exist' + str(e3))
        numEmp= 0
    finally:
        driver.close()
    return numEmp  

def main():
    counter=0  
    finalList = [] 
    finalList.append(['Name','Link','empNumCompany'])
    for link in links:
        counter+=1
        print(counter)
        print(link)    
        templist = []
        try:
            templist.append(link)
            empNumCompany  = fetchEmployeeNumber(link)
            empNumCompanySplit = empNumCompany.split("#")
            templist.append(empNumCompanySplit[0])
            finalList.append(templist)
        except Exception as e:
            templist.append('-NA-')
            finalList.append(templist)
        print(templist)   
    
    # create dataframe from data stored in list of lists and save it
    dataFrame = pd.DataFrame(finalList)
    dataFrame.to_excel('scraped_data.xlsx',index=False)

In [ ]:
main()

In [ ]:
# END